# Time-optimal path planning
This notebook handles standard code for the computation of time-optimal paths in windfields

In [1]:
import os
import sys
import time
import numpy as np
from math import atan, cos, sin
sys.path.extend(['/home/bastien/Documents/work/mermoz', 
                 '/home/bastien/Documents/work/mermoz/src',
                 '/home/bastien/Documents/work/mdisplay',
                 '/home/bastien/Documents/work/mdisplay/src'])

from mermoz.feedback import TargetFB
from mermoz.mdf_manager import MDFmanager
from mermoz.params_summary import ParamsSummary
from mermoz.problem import MermozProblem
from mermoz.model import ZermeloGeneralModel
from mermoz.solver import Solver
from mermoz.stoppingcond import TimedSC, DistanceSC
from mermoz.wind import DiscreteWind
from mermoz.misc import *
from mdisplay.geodata import GeoData

from solver_utils import select_wind

%load_ext autoreload
%autoreload

### Problem definition

Initial point, target, UAV airspeed

In [2]:
# Scale factor
sf = 1.

# Initial point
x_init = sf * np.array((0.6, 0.6))

# Target
x_target = sf * np.array((2.4, 2.4))

# Windfield boundaries
bl = sf * np.array((0.5, 0.5))
tr = sf * np.array((2.5, 2.5))

# UAV airspeed in m/s
v_a = 0.05

case_name = 'double-gyre-kularatne2016'

Set up the problem

In [3]:
# Prepare output directory
output_dir = f'/home/bastien/Documents/work/mermoz/output/example_solver_{case_name}'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Create a file manager to dump problem data
mdfm = MDFmanager()
mdfm.set_output_dir(output_dir)

# Problem coordinates
# coords = COORD_GCS
coords = COORD_CARTESIAN

# Problem wind data
from mermoz.wind import DoubleGyreWind
total_wind = DoubleGyreWind(0.5, 0.5, 2., 2., pi * 0.02)
def domain(x):
    return bl[0] < x[0] < tr[0] and bl[1] < x[1] < tr[1]

# Time window upper bound
# Estimated through great circle distance + 20 percent
T = 40.

# Creates the cinematic model
zermelo_model = ZermeloGeneralModel(v_a, coords=coords)
zermelo_model.update_wind(total_wind)

# Creates the navigation problem on top of the previous model
mp = MermozProblem(zermelo_model, T=T, coords=coords, autodomain=False, domain=domain)

### Solver parameters

In [4]:
u_min = DEG_TO_RAD * 83.
u_max = DEG_TO_RAD * 90.

nt_pmp = 1000
opti_ceil = 0.05
N_dirs = 10
neighb_ceil = 0.9*opti_ceil

solver = Solver(mp,
                x_init,
                x_target,
                T,
                u_min,
                u_max,
                output_dir,
                N_disc_init=N_dirs,
                opti_ceil=opti_ceil,
                neighb_ceil=neighb_ceil,
                n_min_opti=1,
                adaptive_int_step=False,
                N_iter=nt_pmp)

solver.log_config()
solver.setup()

Solve problem

In [5]:
t_start = time.time()
solver.solve_fancy()
t_end = time.time()
time_pmp = t_end - t_start

Shooting 83.0
Shooting 83.8
Shooting 84.6
Shooting 85.3
Shooting 86.1
Shooting 86.9
Shooting 87.7
Shooting 88.4
Shooting 89.2
Shooting 90.0
0.29008, 84.17
0.61177, 83.39
0.46702, 84.94
0.99121, 85.72
1.14922, 86.50
1.23508, 87.28
1.29566, 88.06
1.34327, 88.83
1.38374, 89.61
Iteration 1 - Shooting 84.2 - Depth 1 - Branching LU
Iteration 2 - Shooting 84.4 - Depth 2 - Branching LU
Iteration 3 - Shooting 84.5 - Depth 3 - Branching LU
Iteration 4 - Shooting 84.5 - Depth 4 - Branching LU
Iteration 5 - Shooting 84.5 - Depth 5 - Branching LU
Iteration 6 - Shooting 84.5 - Depth 6 - No branching
Iteration 7 - Shooting 84.5 - Depth 6 - No branching
Iteration 8 - Shooting 84.5 - Depth 5 - Branching LU
Iteration 9 - Shooting 84.5 - Depth 6 - No branching
Iteration 10 - Shooting 84.5 - Depth 6 - No branching
Iteration 11 - Shooting 84.4 - Depth 4 - Branching LU
Iteration 12 - Shooting 84.4 - Depth 5 - Branching LU
Iteration 13 - Shooting 84.4 - Depth 6 - No branching
Iteration 14 - Shooting 84.4 - D

In [5]:
t_start = time.time()
solver.solve_global()
t_end = time.time()
time_pmp = t_end - t_start

    * Pre-processing :  0.000 s
    * Processing      :  4.850 s
    * Total           :  4.850 s


In [17]:
from mermoz.shooting import Shooting
shoot = Shooting(mp.model.dyn, x_init, T,
                             N_iter=nt_pmp, domain=mp.domain, fail_on_maxiter=True, coords=mp.coords)
shoot.set_adjoint(solver.adj_from_dir(1.43309583).reshape((2,)))
traj = shoot.integrate()
mp.trajs = [traj]

### Explicit control laws

In [6]:
sc = DistanceSC(lambda x: np.linalg.norm(x - x_target), opti_ceil)
sc = TimedSC(T)
from mermoz.feedback import FixedHeadingFB, ConstantFB

mp.load_feedback(FixedHeadingFB(mp.model.wind, v_a, 0., mp.coords))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

mp.load_feedback(ConstantFB(0.))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

Dump everything to output directory

In [6]:
nx = 51
ny = 51
mdfm.dump_wind(total_wind, nx=nx, ny=ny, bl=bl, tr=tr)
mdfm.dump_trajs(mp.trajs)
ps = ParamsSummary({}, output_dir)
ps.load_from_solver(solver)
ps.add_param('bl_wind', tuple(bl))
ps.add_param('tr_wind', tuple(tr))
#ps.add_param('nx_wind', nx)
#ps.add_param('ny_wind', ny)
ps.add_param('nt_pmp', nt_pmp)
ps.add_param('pmp_time', time_pmp)
ps.dump()